In [ ]:
# LINEAR
class CustomDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        x = self.data[idx:idx+self.sequence_length]
        y = self.data[idx+self.sequence_length]
        return x, y

train_data = CustomDataset(scaled_array[:n_train-1], sequence_length)
test_data = CustomDataset(scaled_array[n_train:], sequence_length)

# Split train_data into train_set and val_set
train_set, val_set = random_split(train_data, [int(len(train_data) * 0.8), len(train_data) - int(len(train_data) * 0.8)])

train_loader = DataLoader(train_set, batch_size=168, shuffle=True)
val_loader = DataLoader(val_set, batch_size=168, shuffle=False)
test_loader = DataLoader(test_data, batch_size=168, shuffle=False)
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(hidden_size * sequence_length, 50)
        self.fc2 = nn.Linear(50, output_size)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.lstm2(out)
        out = self.flatten(out)
        out = self.fc1(out)
        out = nn.functional.relu(out)
        out = self.fc2(out)
        return out
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lstm = LSTM(n_features, 64, 2, 1).to(device)

criterion = nn.MSELoss()

optimizer = optim.Adam(lstm.parameters(), lr=0.003)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1)

n_epochs = 30

# Train model

train_losses = []
val_losses = []

for epoch in range(n_epochs):
    lstm.train()
    train_loss = 0.0
    
    for X, y in train_loader:
        optimizer.zero_grad()
        y_pred = lstm(X.float())
        loss = criterion(y_pred, y.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    # Update early stopping
    # Update learning rate scheduler
    scheduler.step(train_loss)

    # Check if early stopping condition is met
    if scheduler.state_dict()['_last_lr'][0] < 1e-4:
        print("Early stopping")
        break

    # Evaluate model
    # Evaluate on validation set for early stopping
    lstm.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X, y in val_loader:
            y_pred = lstm(X.float())
            loss = criterion(y_pred, y.float())
            val_loss += loss.item()
        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        print(f"Epoch {epoch+1} - Training Loss: {train_loss:.6f}; Validation Loss: {val_loss:.6f}")

    # Update learning rate scheduler
    scheduler.step(train_loss)

    # Check if early stopping condition is met
    if scheduler.state_dict()['_last_lr'][0] < 1e-4:
        print("Early stopping")
        break

# Evaluate model
lstm.eval()
predictions = []
truth = []
#
with torch.no_grad():
    for X, y in test_loader:
        y_pred = lstm(X.float())
        predictions.append(y_pred.detach().numpy())
        truth.append(y.detach().numpy())
predictions = np.concatenate(predictions).reshape(-1, 1)
truth = np.concatenate(truth).reshape(-1, 1)

predictions_scaled = target_scaler.inverse_transform(predictions)

truth_scaled = target_scaler.inverse_transform(truth)

rmse = mean_squared_error(truth_scaled, predictions_scaled, squared=False)

print(f"Test RMSE: {rmse:.6f}")
